## 複数の画像特徴を比較するnotebook
* ノートブックの趣旨
> - PetFinderコンペのデータが題材
> - 複数の学習済みモデルを用いて特徴量を作成し、LightGBMの説明変数とした時の精度を比較する
* 試した学習済みモデル
> - keras : densenet121
> - keras : inceptionresnetv2
> - keras : nannetlarge














In [0]:
!pip install optuna

In [0]:
import json
import scipy as sp
import pandas as pd
import numpy as np

from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from collections import Counter

# import lightgbm as lgb
import optuna.integration.lightgbm as lgb
import lightgbm as lgb_original

BEST_PARAMS = []

## 学習用関数の定義

In [0]:
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

## 必要データの読み込み

In [0]:
%%time

COMPETITION_FOLDER = "/content/drive/My Drive/20200515_マルチモーダル検証/data"

print('Train')
train = pd.read_csv(COMPETITION_FOLDER + "/train.csv")
print(train.shape)

print('Test')
test = pd.read_csv(COMPETITION_FOLDER + "/test.csv")
print(test.shape)

print('Breeds')
breeds = pd.read_csv(COMPETITION_FOLDER + "/breed_labels.csv")
print(breeds.shape)

print('Colors')
colors = pd.read_csv(COMPETITION_FOLDER + "/color_labels.csv")
print(colors.shape)

print('States')
states = pd.read_csv(COMPETITION_FOLDER + "/state_labels.csv")
print(states.shape)

Train
(14993, 24)
Test
(3972, 23)
Breeds
(307, 3)
Colors
(7, 2)
States
(15, 2)
CPU times: user 122 ms, sys: 8.97 ms, total: 131 ms
Wall time: 148 ms


## 比較を実行する

In [0]:
IMAGE_FEAT_FOLDER = "/content/drive/My Drive/20200515_マルチモーダル検証/image_features"

sample_list = [["/train_img_features_densenet121.csv", 
                "/test_img_features_densenet121.csv", 
                "Dense121"],
              ["/train_img_features_densenet121_svd32.csv", 
               "/test_img_features_densenet121_svd32.csv", 
               "Dense121_svd32"],
              ["/train_img_features_inseption_resnet_v2.csv", 
               "/test_img_features_inseption_resnet_v2.csv", 
               "Inception_ResNetv2"],
              ["/train_img_features_inseption_resnet_v2_svd32.csv", 
               "/test_img_features_inseption_resnet_v2_svd32.csv", 
               "Inception_ResNetv2_svd32"],
               ["/train_img_features_nasnet.csv",
                "/test_img_features_nasnet.csv",
                "nasnet_large"],
               ["/train_img_features_nasnet_svd32.csv",
                "/test_img_features_nasnet_svd32.csv",
                "nasnet_large"]
               ]

In [0]:
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb

In [0]:
train_df_, valid_df_ = train_test_split(train, test_size = 0.2)

In [0]:
train_df.shape, valid_df.shape

((11994, 24), (2999, 24))

In [0]:
rmse_train_list = []
rmse_valid_list = []


for pair in sample_list:

    train_df = train_df_.copy()
    valid_df = valid_df_.copy()
    
    # 画像特徴データの読み込み
    image_feat_train = pd.read_csv(IMAGE_FEAT_FOLDER + pair[0]).rename(columns={"Unnamed: 0":"PetID"})
    
    # 画像特徴を結合する
    train_df = train_df.merge(image_feat_train, on="PetID", how="left")
    valid_df = valid_df.merge(image_feat_train, on="PetID", how="left")
    
    # 不要になった列を削除する
    train_df.drop(['Name', 'RescuerID', 'Description', 'PetID'], axis=1, inplace=True)
    valid_df.drop(['Name', 'RescuerID', 'Description', 'PetID'], axis=1, inplace=True)
    
    numeric_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt']+ list(train_df)[19:]
    cat_cols = list(set(train_df.columns) - set(numeric_cols))
    train_df.loc[:, cat_cols] = train_df[cat_cols].astype('category')
    valid_df.loc[:, cat_cols] = valid_df[cat_cols].astype('category')

    d_train = lgb_original.Dataset(train_df.drop("AdoptionSpeed", axis=1), label=train_df["AdoptionSpeed"])
    d_valid = lgb_original.Dataset(valid_df.drop("AdoptionSpeed", axis=1), label=valid_df["AdoptionSpeed"])
    watchlist = [d_train, d_valid]
    print('Train LGB')
    best_params, history = {}, []

    params = {'application': 'regression',
             'boosting': 'gbdt',
             'metric': 'rmse',
              'verbosity': -1,
              'data_random_seed': 3,
              'early_stop': 100,
              'verbose_eval': 1000,
              'num_rounds': 10000}

    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=10000,
                      valid_sets=watchlist,
                      verbose_eval=100,
                      best_params=best_params,
                      tuning_history=history,
                      early_stopping_rounds=50)
    
    model_2 = lgb_original.train(best_params,
                  train_set=d_train,
                  num_boost_round=10000,
                  valid_sets=watchlist,
                  verbose_eval=100,
                  early_stopping_rounds=50)

    pred = model_2.predict(train_df.drop("AdoptionSpeed", axis=1))
    score = rmse(pred, train_df["AdoptionSpeed"])
    rmse_train_list.append(score)

    pred = model_2.predict(valid_df.drop("AdoptionSpeed", axis=1))
    score = rmse(pred, valid_df["AdoptionSpeed"])
    rmse_valid_list.append(score)


    

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

train is experimental (supported from v0.18.0). The interface can change in the future.




  0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Train LGB


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_rounds` in params. Will use it instead of argument

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 50 rounds.





feature_fraction, val_score: 1.070827:   0%|          | 0/7 [00:05<?, ?it/s]


feature_fraction, val_score: 1.070827:  14%|#4        | 1/7 [00:05<00:31,  5.24s/it][I 2020-05-17 07:07:06,749] Finished trial#0 with value: 1.0708269737017382 with parameters: {'feature_fraction': 0.7}. Best is trial#0 with value: 1.0708269737017382.



feature_fraction, val_score: 1.070827:  14%|#4        | 1/7 [00:05<00:31,  5.24s/it]

...




min_data_in_leaf, val_score: 1.069968:  60%|######    | 3/5 [00:02<00:01,  1.51it/s]


min_data_in_leaf, val_score: 1.069968:  80%|########  | 4/5 [00:02<00:00,  1.51it/s][I 2020-05-17 08:04:48,720] Finished trial#66 with value: 1.0779631225828303 with parameters: {'min_child_samples': 10}. Best is trial#65 with value: 1.0744745642528948.



min_data_in_leaf, val_score: 1.069968:  80%|########  | 4/5 [00:02<00:00,  1.51it/s]

[100]	training's rmse: 0.921294	valid_1's rmse: 1.07834
Early stopping, best iteration is:
[61]	training's rmse: 0.969332	valid_1's rmse: 1.07796
Training until validation scores don't improve for 50 rounds.





min_data_in_leaf, val_score: 1.069968:  80%|########  | 4/5 [00:03<00:00,  1.51it/s]


min_data_in_leaf, val_score: 1.069968: 100%|##########| 5/5 [00:03<00:00,  1.47it/s]

[100]	training's rmse: 0.935833	valid_1's rmse: 1.07681
Early stopping, best iteration is:
[69]	training's rmse: 0.968008	valid_1's rmse: 1.07534


Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 0.925087	valid_1's rmse: 1.0719
Early stopping, best iteration is:
[77]	training's rmse: 0.950449	valid_1's rmse: 1.06997


In [0]:
rmse_train_list

[0.7198132468977999,
 0.8823165019437892,
 0.9404288158936785,
 0.9199973395766876,
 0.7678307753808918,
 0.950448602920285]

In [0]:
rmse_valid_list

[1.0644118012143422,
 1.0648955309008046,
 1.0657640939424513,
 1.0636611987836975,
 1.0736521025567367,
 1.0699508579199097]